In [26]:
#  this notebook showcases multi-threading to query the objective functions!
import torch as tr
# import scripts
from src.api_helper import env
import concurrent.futures
import multiprocessing
dtype=tr.float32

##### data

In [27]:
env.rosenbrock(tr.tensor([1,2],dtype=dtype))  # query the objective functions

tensor([100.])

In [44]:
n = 10000

In [45]:
def thread_function_io(name, query):
    print("Thread %s: starting", name)
    env.rosenbrock(query)
    print("Thread %s: finishing", name)

def thread_function_cpu(query):
    # print("Thread %s: starting", name)
    env.rosenbrock(query)
    # print("Thread %s: finishing", name)

names = [f"thread_{i}" for i in range(n)]
q = [tr.tensor([1,2],dtype=dtype)] * n

In [46]:
%%time
# concurrent.futures.ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # context 
    executor.map(thread_function_io, zip(names, q))

CPU times: user 486 ms, sys: 20.1 ms, total: 506 ms
Wall time: 497 ms


In [47]:
%%time
with multiprocessing.Pool(processes=3) as pool:  # because this is cpu bound 
    pool.map(thread_function_cpu,  q)

CPU times: user 23.2 ms, sys: 38.6 ms, total: 61.8 ms
Wall time: 416 ms


In [49]:
%%time 
for i in range(n):
    env.rosenbrock(q[i])

CPU times: user 897 ms, sys: 0 ns, total: 897 ms
Wall time: 899 ms


In [50]:
q = [tr.rand(2,) for _ in range(5)]
q

[tensor([0.1384, 0.8872]),
 tensor([0.9864, 0.0389]),
 tensor([0.8725, 0.6620]),
 tensor([0.3451, 0.3976]),
 tensor([0.0514, 0.3934])]

In [51]:
a = tr.zeros(2,)
for i in q:
    a = tr.cat([a,i])
a = a[2:].view(-1,2)
a


tensor([[0.1384, 0.8872],
        [0.9864, 0.0389],
        [0.8725, 0.6620],
        [0.3451, 0.3976],
        [0.0514, 0.3934]])

In [52]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # context 
    ans = executor.map(env.rosenbrock,  q)

In [53]:
for i in ans:
    print(i)

tensor([76.0850])
tensor([87.2680])
tensor([1.0024])
tensor([8.1851])
tensor([16.1666])


In [54]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # context 
    ans = executor.map(env.rosenbrock,  a)

In [55]:
for i in ans:
    print(i)

tensor([76.0850])
tensor([87.2680])
tensor([1.0024])
tensor([8.1851])
tensor([16.1666])


In [56]:
with multiprocessing.Pool(processes=3) as pool:  # because this is cpu bound 
    ans = pool.map(env.rosenbrock,  a)

In [57]:
for i in ans:
    print(i)

tensor([76.0850])
tensor([87.2680])
tensor([1.0024])
tensor([8.1851])
tensor([16.1666])
